In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import fsspec
import xarray as xr
import carbonplan_trace.v1.glas_height_metrics as ht
from carbonplan_trace.v1.glas_preprocess import calculate_derived_variables

In [ ]:
from carbonplan_trace.v1.glas_extract import (
    extract_GLAH14_data,
    extract_GLAH01_data,
)

# test extraction combined data

f01 = "/home/jovyan/data/glas/example/GLAH01_033_2107_003_0241_4_02_0001.H5"
f14 = "/home/jovyan/data/glas/example/GLAH14_634_2107_003_0239_0_01_0001.H5"

data01 = extract_GLAH01_data(f01).chunk(
    {"record_index": 1000, "shot_number": 10}
)
data14 = extract_GLAH14_data(f14).chunk(
    {"record_index": 1000, "shot_number": 10}
)

combined = data14.merge(data01, join="inner")
combined = combined.chunk({"record_index": 1000, "shot_number": 10})

In [ ]:
# variables used in the rest of the preprocess
combined = calculate_derived_variables(combined)

# stack the record index and shot number together so we have a ~1D tabular data structure
combined = combined.stack(unique_index=("record_index", "shot_number"))


def convert_long3_to_long1(long3):
    # see https://confluence.ecmwf.int/pages/viewpage.action?pageId=149337515
    long1 = (long3 + 180) % 360 - 180
    return long1


combined["lon"] = convert_long3_to_long1(combined.lon)

In [ ]:
combined

In [ ]:
srtm = (
    xr.open_rasterio(
        #     '/home/jovyan/data/N40W078.hgt',
        "/home/jovyan/data/N42W078.hgt",
        parse_coordinates=True,
    )
    .rename({"x": "lon", "y": "lat"})
    .squeeze(drop=True)
)

In [ ]:
combined["SRTM"] = srtm.sel(
    lat=combined.lat, lon=combined.lon, method="nearest"
).drop_vars(["lat", "lon"])

In [ ]:
df = combined[
    [
        "lat",
        "lon",
        "elevation",
        "elevation_correction",
        "elevation_SRTM",
        "delta_ellipse",
        "geoid",
        "ground_peak_dist",
        "elv_cloud_flg",
        "elev_use_flg",
        "SRTM",
    ]
].to_dataframe()

In [ ]:
len(df)

In [ ]:
df = df.loc[~df.lat.isnull()]

In [ ]:
len(df)

In [ ]:
df["glas_elev"] = df.elevation + df.elevation_correction

In [ ]:
df.elevation_SRTM.isnull().sum()

In [ ]:
not_consistent_1 = (
    df.glas_elev - df.elevation_SRTM.fillna(df.glas_elev)
).abs() > 30
not_consistent_1.mean()

In [ ]:
box = (
    (df.lat >= srtm.lat.min().values)
    & (df.lat <= srtm.lat.max().values)
    & (df.lon >= srtm.lon.min().values)
    & (df.lon <= srtm.lon.max().values)
)
df["glas_SRTM_adj"] = df.elevation_SRTM - df.delta_ellipse - df.geoid
df["SRTM_in_TP"] = df.SRTM + df.delta_ellipse + df.geoid

In [ ]:
(df.loc[box].SRTM - df.loc[box].elevation_SRTM).mean()

In [ ]:
(df.loc[box].SRTM - df.loc[box].glas_SRTM_adj).mean()

In [ ]:
(df.loc[box].SRTM_in_TP - df.loc[box].elevation_SRTM).mean()

In [ ]:
df.loc[box].isnull().sum()

In [ ]:
not_consistent_2 = (df.loc[box].glas_elev - df.loc[box].SRTM_in_TP).abs() > 30
not_consistent_2.mean()

In [ ]:
not_consistent_2 = (
    df.loc[box].glas_elev - df.loc[box].elevation_SRTM
).abs() > 30
not_consistent_2.mean()

In [ ]:
df.loc[box, ["glas_elev", "elevation_SRTM", "SRTM_in_TP"]]